In [ ]:
import psycopg2
import csv
import os
import time
import tarfile
from dotenv import load_dotenv

load_dotenv()

In [ ]:
table_name = "recordings"
from_date = "2023-04-10"
to_date = "2023-06-14"
date_field = "recorded"
page_size = 50000
total_count = 100000

In [ ]:
db_host = os.getenv('DATABASE_HOST')
db_name = os.getenv('DATABASE_NAME')
db_user = os.getenv('DATABASE_USER')
db_password = os.getenv('DATABASE_PASSWORD')

connection = psycopg2.connect(
    host=db_host,
    database=db_name,
    user=db_user,
    password=db_password
)

In [ ]:
count_query = f"SELECT COUNT(*) FROM {table_name} WHERE {date_field} >= '{from_date}' AND {date_field} < '{to_date}'"
count_cursor = connection.cursor()
count_cursor.execute(count_query)
total_count = count_cursor.fetchone()[0]

print("Total records:", total_count)

In [ ]:
num_pages = total_count // page_size
if total_count % page_size > 0:
    num_pages += 1

for i in range(num_pages):

    offset = i * page_size
    paging_query = f"SELECT * FROM {table_name} WHERE {date_field} >= '{from_date}' AND {date_field} < '{to_date}' LIMIT {page_size} OFFSET {offset}"
    
    paging_cursor = connection.cursor()
    paging_cursor.execute(paging_query)
    
    records = paging_cursor.fetchall()

    current_file_path = f'output/backup_{i+1}.csv'

    with open(current_file_path, 'a', newline='') as csv_file:
        writer = csv.writer(csv_file)

        isEmpty = csv_file.tell() == 0
        if isEmpty:
            column_names = []
            for desc in paging_cursor.description:
                column_names.append(desc[0])
            writer.writerow(column_names)

        for r in records:
            writer.writerow(r)

    percentage = f'{((i+1) * 100) / num_pages:.0f}%'
    print(f'Writing {len(records)} records to CSV file ({percentage})')
    
    time.sleep(0.2)

In [34]:
dir_path = './output/'
tar_name = './output/compressed_csvs.tar.gz'

with tarfile.open(tar_name, 'w:gz') as tar:
    for filename in os.listdir(dir_path):
        if filename.endswith('.csv'):
            csv_path = os.path.join(dir_path, filename)

            tar.add(csv_path, arcname=os.path.basename(csv_path))
            
print('Compressed CSV files into tar.gz file')

In [ ]:
count_cursor.close()
paging_cursor.close()
connection.close()